In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import Demand
import Solar
import Price
import Date

In [2]:
def analysis(bigdata):

    eff = 0.9
    batterycharge = 0
    chargeneeded = 0
    median = np.median(bigdata['Grid Buy Price'])
    

    for n in range(0,24):
        
        if bigdata['Net Demand'][n] <= 0:
            batterycharge = batterycharge - bigdata['Net Demand'][n]*eff
            
        if bigdata['Grid Buy Price'][n] >= median*1.5:
            chargeneeded = chargeneeded + bigdata['Net Demand'][n]/eff
            

    return batterycharge, chargeneeded


In [3]:
## method to run the SMART system ##

def run_SMART(bigdata):
    
    batteryoutput = np.zeros(24)
    soc = np.ones(24)*0
    capacity = 750
    power = 420
    eff = 0.9
    
    median = np.median(bigdata['Grid Buy Price'])
    predicted_battery_charge, required_battery_charge = analysis(bigdata)

    for n in range(0,24):
        
        if n == 0:
            soc_temp = 0
        else:
            soc_temp = soc[n-1]

        ## the smart design ##
        ## the smart design ##
        ## the smart design ##

        
        if required_battery_charge > predicted_battery_charge: 
            if n == 5:
                batteryoutput[n] = max(-power, -(1/eff)*(required_battery_charge - predicted_battery_charge))
                soc[n] = soc_temp - batteryoutput[n]*(1/eff)
                continue
                
        if n == 0:
            use = False
        if bigdata['Grid Buy Price'][n] >= median*1.5:
            use = True
            

        ## the smart design ##
        ## the smart design ##
        ## the smart design ##

        if bigdata['Net Demand'][n] > 0:   #using the battery
            batteryoutput[n] = min(bigdata['Net Demand'][n], soc_temp*eff, power)
            soc[n] = soc_temp - batteryoutput[n]*(1/eff)
            if use == False:
                soc[n] = soc[n-1]
                batteryoutput[n] = 0

        elif bigdata['Net Demand'][n] < 0:     # charge the battery
            batteryoutput[n] = max(bigdata['Net Demand'][n], -(1/eff)*(capacity - soc_temp), -power)
            soc[n] = soc_temp - batteryoutput[n]*(eff)

        elif bigdata['Net Demand'][n] == 0:      # do nothing
            soc[n] = soc_temp

        
    ## adds the battery output to total demand and adds to bigdata ##

    bigdata['Net Demand - SMART'] = bigdata['Net Demand']-batteryoutput
    bigdata['Running Cost - SMART'] = bigdata['Net Demand - SMART']*bigdata['Grid Buy Price']
    bigdata['Running Cost - SMART'] = bigdata['Running Cost - SMART'].clip(lower=0)
    bigdata['Running Profit - SMART'] = -bigdata['Net Demand - SMART']*bigdata['Grid Sell Price']
    bigdata['Running Profit - SMART'] = bigdata['Running Profit - SMART'].clip(lower=0)
    bigdata['soc - SMART'] = soc
    bigdata['battery output - SMART'] = batteryoutput
    
    #bigdata = bigdata.round(3)
    ## returns the new bigdata ##
    
    return bigdata


In [4]:
## method to run the battery ##

def run_battery(bigdata):
    
    batteryoutput = np.zeros(24)
    soc = np.ones(24)*0
    capacity = 750
    power = 420
    eff = 0.9


    for n in range(0,24):
        if n == 0:
            soc_temp = 0
        else:
            soc_temp = soc[n-1]

        if bigdata['Net Demand'][n] > 0:   #using the battery
            batteryoutput[n] = min(bigdata['Net Demand'][n], soc_temp*eff, power)
            soc[n] = soc_temp - batteryoutput[n]*(1/eff)

        elif bigdata['Net Demand'][n] < 0:     # charge the battery, NOTE NET DEMAND = -VE
            batteryoutput[n] = max(bigdata['Net Demand'][n], -(1/eff)*(capacity - soc_temp), -power)
            soc[n] = soc_temp - batteryoutput[n]*(eff)

        elif bigdata['Net Demand'][n] == 0:      # do nothing
            soc[n] = soc_temp


    ## adds the battery output to total demand and adds to bigdata ##

    bigdata['Net Demand - battery'] = bigdata['Net Demand']-batteryoutput
    bigdata['Running Cost - battery'] = bigdata['Net Demand - battery']*bigdata['Grid Buy Price']
    bigdata['Running Cost - battery'] = bigdata['Running Cost - battery'].clip(lower=0)
    bigdata['Running Profit - battery'] = -bigdata['Net Demand - battery']*bigdata['Grid Sell Price']
    bigdata['Running Profit - battery'] = bigdata['Running Profit - battery'].clip(lower=0)
    bigdata['soc - battery'] = soc
    bigdata['battery output - battery'] = batteryoutput
    
    #bigdata = bigdata.round(3)
    ## returns the new bigdata ##
    
    return bigdata

In [5]:
## initialises the daily cost vectors ##
SMART_cost = np.zeros(365)
SMART_profit = np.zeros(365)
_import_ = np.zeros(365)
_export_ = np.zeros(365)

for n in range(0, 365):

    ## loads in the data ##

    daywanted = n+1
    datewanted = Date.getdatewanted(daywanted)
    totaldemand = Demand.gettotaldemand(daywanted)
    solarsupply = Solar.getsolarsupply(daywanted)
    buyprice = Price.getconstantbuyprice(daywanted)
    sellprice = Price.getconstantsellprice(daywanted)
    bigdata =pd.concat([totaldemand, solarsupply, buyprice, sellprice], axis=1)
    bigdata['Net Demand'] = bigdata['Total Demand (kWh)']-bigdata['Total Supply (kWh)']
    bigdata['Running Cost'] = bigdata['Grid Buy Price']*bigdata['Net Demand']
    bigdata['Running Cost'] = bigdata['Running Cost'].clip(lower=0)
    bigdata['Running Profit'] = -bigdata['Grid Sell Price']*bigdata['Net Demand']
    bigdata['Running Profit'] = bigdata['Running Profit'].clip(lower=0)

    ## runs the battery for that day ##

    bigdata = run_SMART(bigdata)

    ## adds the day into the year vector ##

 
    SMART_cost[n] = bigdata['Running Cost - SMART'].sum()
    SMART_profit[n] = bigdata['Running Profit - SMART'].sum()
    
    _import_[n] = bigdata['Net Demand - SMART'].clip(lower=0).sum()
    temp = bigdata['Net Demand - SMART']*(-1)
    _export_[n] = temp.clip(lower=0).sum()


In [6]:
SMART_cost.sum()

52012.91232

In [7]:
SMART_profit.sum()

67416.66873333334

In [8]:
SMART_profit.sum()-SMART_cost.sum()

15403.756413333336

In [9]:
_import_.sum()

361200.78

In [10]:
_export_.sum()

1225757.6133333333

In [11]:
_export_.sum()-_import_.sum()

864556.8333333333